# Lecture 16 

## Reinforcement Learning

We begin our formal study of reinforcement learning (RL) by examining the mathematical structure of **memory-less sequential decision algorithms**, which correspond to the class of **Markov Decision Processes (MDPs)** with **stationary policies** and **no explicit memory of past states or actions**. These systems provide a mathematically tractable and conceptually foundational framework upon which more general reinforcement learning methods are built.

Let us define the concept of a *Markov Decision Process*. A Markov Decision Process is a 5-tuple

$$
\mathcal{M} = (S, A, P, R, \gamma),
$$

where:

* $S$ is a finite set of **states**.
* $A$ is a finite set of **actions**.
* $P: S \times A \times S \to [0,1]$ is the **state transition probability function**, so that for $s, s' \in S$ and $a \in A$, we write

  $$
  P(s' \mid s, a) = \mathbb{P}(s_{t+1} = s' \mid s_t = s, a_t = a).
  $$
* $R: S \times A \to \mathbb{R}$ is the **reward function**, where $R(s,a)$ denotes the expected immediate reward received when action $a$ is taken in state $s$.
* $\gamma \in [0,1)$ is the **discount factor**, which determines the present value of future rewards and is used to guarantee convergence of infinite-horizon reward sums.

The *Markov* property asserts that the transition probability distribution of the next state depends only on the current state and action, and not on the sequence of events that preceded it. Formally, for all $t \in \mathbb{N}$,

$$
\mathbb{P}(s_{t+1} = s' \mid s_0, a_0, s_1, a_1, \dots, s_t = s, a_t = a) = \mathbb{P}(s_{t+1} = s' \mid s_t = s, a_t = a).
$$

A **policy** is a rule that determines the agent’s behavior. In the memory-less (also called *stationary*) case, a policy is a function

$$
\pi: S \to A
$$

that maps each state deterministically to an action. More generally, one can consider stochastic policies, which are functions $\pi: S \times A \to [0,1]$ satisfying $\sum_{a \in A} \pi(a \mid s) = 1$ for all $s \in S$.

The objective in this setting is to identify a policy $\pi$ that maximizes the expected cumulative discounted reward over an infinite time horizon. That is, for a given initial state $s_0$, we define the **value function** of a policy $\pi$ by

$$
V^\pi(s) = \mathbb{E}_\pi\left[ \sum_{t=0}^\infty \gamma^t R(s_t, \pi(s_t)) \mid s_0 = s \right],
$$

where the expectation is taken over trajectories $(s_0, s_1, s_2, \dots)$ generated by the transition probabilities $P$ under the policy $\pi$. In turn, we define the **optimal value function** as

$$
V^*(s) = \sup_\pi V^\pi(s),
$$

and a policy $\pi^*$ is said to be *optimal* if $V^{\pi^*}(s) = V^*(s)$ for all $s \in S$. The foundational recursive characterization of the value function is given by the **Bellman equation**. For a fixed policy $\pi$, the Bellman equation reads:

$$
V^\pi(s) = R(s, \pi(s)) + \gamma \sum_{s' \in S} P(s' \mid s, \pi(s)) V^\pi(s').
$$

This is a system of $|S|$ linear equations in the unknowns ${V^\pi(s)}_{s \in S}$ and has a unique solution under the assumption $\gamma < 1$.  The optimal value function $V^*$ satisfies the **Bellman optimality equation**:

$$
V^*(s) = \max_{a \in A} \left[ R(s,a) + \gamma \sum_{s' \in S} P(s' \mid s, a) V^*(s') \right].
$$

This equation is nonlinear due to the $\max$ operator, and its solution characterizes the optimal policy by:

$$
\pi^*(s) = \arg\max_{a \in A} \left[ R(s,a) + \gamma \sum_{s' \in S} P(s' \mid s, a) V^*(s') \right].
$$

The study of memory-less policies in this context is not merely pedagogical. Under mild regularity conditions, it is known that **there exists an optimal deterministic stationary policy**, even when the state and action spaces are finite but large. This result is nontrivial and arises from the structure of the Bellman optimality operator as a contraction mapping in the sup-norm, a property which allows one to employ **value iteration** or **policy iteration** algorithms to converge to the optimal policy.

This setting forms the foundation of much of modern reinforcement learning, even when explicit models of the transition probabilities and rewards are not available, leading to approximate and sample-based algorithms. Before moving on to such methods, it is essential to understand the exact, model-based solution strategies in the fully known MDP setting.


### An Example: A Three-State Grid World

Let us define a toy environment consisting of three states arranged linearly:

$$
\mathcal{S} = \{s_1, s_2, s_3\},
$$

with a single agent moving left or right between them. The possible actions are

$$
\mathcal{A} = \{\text{Left}, \text{Right}\}.
$$

We assume the agent begins in state $s_2$. The environment behaves according to the following deterministic transition rules:

* From $s_1$, the action *Left* keeps the agent in $s_1$, and *Right* moves it to $s_2$.
* From $s_2$, *Left* moves the agent to $s_1$, and *Right* moves it to $s_3$.
* From $s_3$, *Right* keeps the agent in $s_3$, and *Left* moves it to $s_2$.

This defines a deterministic transition function $P : \mathcal{S} \times \mathcal{A} \to \mathcal{S}$.

We define the reward function as follows:

$$
R(s, a) = \begin{cases}
+1 & \text{if } s = s_3 \text{ and } a = \text{Right}, \\
0 & \text{otherwise}.
\end{cases}
$$

That is, the only positive reward is obtained by choosing *Right* in $s_3$, even though this action does not change the state. All other actions yield zero reward.

![A graphical representation of the MDP](../images/markov-decision.png)

The goal of the agent is to find an optimal policy $\pi^\ast$ that maximizes the expected cumulative reward. Let us assume a discount factor $\gamma = 0.9$. Let us fix a specific policy:

$$
\pi(s_1) = \text{Right},\quad \pi(s_2) = \text{Right},\quad \pi(s_3) = \text{Right}.
$$

That is, the agent always moves to the right if possible. Under this policy, the state transitions are:

* $s_1 \xrightarrow{\text{Right}} s_2$
* $s_2 \xrightarrow{\text{Right}} s_3$
* $s_3 \xrightarrow{\text{Right}} s_3$

We now solve for $V^\pi(s)$.

#### Step 1: Bellman equations

Let us denote:

$$
v_1 = V^\pi(s_1),\quad v_2 = V^\pi(s_2),\quad v_3 = V^\pi(s_3).
$$

From the policy and transition rules:

* $v_1 = R(s_1,\text{Right}) + \gamma V^\pi(s_2) = 0 + \gamma v_2$
* $v_2 = R(s_2,\text{Right}) + \gamma V^\pi(s_3) = 0 + \gamma v_3$
* $v_3 = R(s_3,\text{Right}) + \gamma V^\pi(s_3) = 1 + \gamma v_3$

#### Step 2: Solve recursively

From the third equation:

$$
v_3 = 1 + \gamma v_3 \Rightarrow v_3(1 - \gamma) = 1 \Rightarrow v_3 = \frac{1}{1 - \gamma} = \frac{1}{0.1} = 10.
$$

Then:

$$
v_2 = \gamma v_3 = 0.9 \cdot 10 = 9, \\
v_1 = \gamma v_2 = 0.9 \cdot 9 = 8.1.
$$

Thus, under this policy $\pi$, the value function is:

$$
V^\pi(s_1) = 8.1,\quad V^\pi(s_2) = 9,\quad V^\pi(s_3) = 10.
$$

This illustrates how the reward structure and transition dynamics induce value across the state space, even in states where no reward is directly received, due to their proximity to a rewarding state under the policy.

### A Python Implementation for the Computation

In [1]:
import numpy as np
import mdptoolbox
from collections import deque, defaultdict
from typing import List, Tuple

In [13]:
# Define the states and actions
states = ['s1', 's2', 's3']
actions = ['Left', 'Right']

# Define deterministic transition function P(s, a) -> s'
def transition(state, action):
    if state == 's1':
        return 's1' if action == 'Left' else 's2'
    elif state == 's2':
        return 's1' if action == 'Left' else 's3'
    elif state == 's3':
        return 's2' if action == 'Left' else 's3'

# Define reward function R(s, a)
def reward(state, action):
    return 1 if state == 's3' and action == 'Right' else 0

In [14]:
# Define discount factor
gamma = 0.9  

# Define the fixed policy π(s) = 'Right' for all s
policy = {s: 'Right' for s in states}

# Initialize value function
V = {s: 0.0 for s in states}

# Iterative policy evaluation
tolerance = 1e-6
delta = float('inf')
iteration = 0

while delta > tolerance:
    delta = 0.0
    new_V = V.copy()
    for s in states:
        a = policy[s]
        s_prime = transition(s, a)
        r = reward(s, a)
        new_V[s] = r + gamma * V[s_prime]
        delta = max(delta, abs(new_V[s] - V[s]))
    V = new_V
    iteration += 1

# Print results
print(f"Converged after {iteration} iterations")
{k: np.round(v,3) for k,v in V.items()}

Converged after 133 iterations


{'s1': np.float64(8.1), 's2': np.float64(9.0), 's3': np.float64(10.0)}

### Optimality

One may then ask whether this policy is optimal. In this case, the only positive reward is obtained by looping in $s_3$ under action *Right*. Since $s_3$ is absorbing under *Right*, and all rewards elsewhere are zero, any policy that leads to $s_3$ and remains there taking *Right* forever will be optimal.

Hence, the above policy $\pi$ is indeed optimal. The **optimal value function** $V^\ast$ is therefore identical to the one we computed:

$$
V^*(s_1) = 8.1,\quad V^*(s_2) = 9,\quad V^*(s_3) = 10.
$$

This explicit example shows how a discrete deterministic MDP with a simple reward structure and transition model can be solved analytically, and how Bellman’s equations enable recursive computation of value functions.

## Another Example: One Dimensional Solo Game

Let us do another a step-by-step solution of the **Bellman equation**. This time our game is a deterministic single-player game inspired by [Solo (a variant of peg solitaire)](https://en.wikipedia.org/wiki/Peg_solitaire), but set in a **1D array of 10 bins**. We will treat this as a deterministic episodic environment terminating when no valid moves remain. The game’s state space will be a subset of ${0,1}^{10}$, encoding whether a bin is filled ($1$) or empty ($0$).

### Step 1: Define the Game Environment

We define the game as a deterministic, acyclic, single-agent transition system on binary strings of length $n = 10$, representing the **occupancy status** of 10 contiguous bins.

Let the state space be

$$
\mathcal{S} \subseteq \{0,1\}^{10},
$$

where a state $s = (s_0, \dots, s_9) \in \mathcal{S}$ represents the presence ($s_i = 1$) or absence ($s_i = 0$) of a peg at bin $i$.

#### Initial State

We define the **initial state** as:

$$
s^{(0)} = (1,1,1,1,0,1,1,1,1,1),
$$

i.e., all bins are filled except the middle bin (index 4).

#### Legal Moves

The allowed **moves** correspond to standard **peg solitaire jumps**:

1. **Rightward jump:** If there exists an index $i$ such that $(s_i, s_{i+1}, s_{i+2}) = (1,1,0)$, then a jump from $i$ over $i+1$ to $i+2$ results in:

   $$
   s_i \gets 0,\quad s_{i+1} \gets 0,\quad s_{i+2} \gets 1
   $$

2. **Leftward jump:** If $(s_i, s_{i+1}, s_{i+2}) = (0,1,1)$, then a jump from $i+2$ over $i+1$ to $i$ results in:

   $$
   s_i \gets 1,\quad s_{i+1} \gets 0,\quad s_{i+2} \gets 0
   $$

Each legal move produces a new deterministic state $s' \in \mathcal{S}$. We assume that **only one jump is allowed per time step**.

#### Terminal States

A state $s \in \mathcal{S}$ is called **terminal** if **no valid jump is possible**:

$$
\forall i \in \{0, \dots, 7\},\quad (s_i, s_{i+1}, s_{i+2}) \notin \{(1,1,0), (0,1,1)\}
$$

#### Reward Function

We define a scalar reward function $R: \mathcal{S} \to \mathbb{R}$ as $ R(s) = -\sum_{i=0}^9 s_i$ if $s$ is a terminal state, and $0$ otherwise.

In [15]:
State = Tuple[int, ...]

def get_valid_moves(state: State) -> List[State]:
    state_array = np.array(state)
    n = len(state)
    moves = []
    # Rightward jumps: (1,1,0) → (0,0,1)
    for i in range(n - 2):
        if state[i] == 1 and state[i+1] == 1 and state[i+2] == 0:
            new_state = state_array.copy()
            new_state[i], new_state[i+1], new_state[i+2] = 0, 0, 1
            moves.append(tuple(new_state))
    # Leftward jumps: (0,1,1) → (1,0,0)
    for i in range(n - 2):
        if state[i] == 0 and state[i+1] == 1 and state[i+2] == 1:
            new_state = state_array.copy()
            new_state[i], new_state[i+1], new_state[i+2] = 1, 0, 0
            moves.append(tuple(new_state))
    return moves

def is_terminal(state: State) -> bool:
    return len(get_valid_moves(state)) == 0

def reward(state: State) -> int:
    if is_terminal(state):
        return -sum(state)  # Fewer pegs → better
    else:
        return 0

def render(state: State) -> str:
    return ''.join('●' if x else '○' for x in state)

In [46]:
initial_state: State = (1, 1, 1, 1, 0, 1, 1, 1, 1, 1)

print("Initial state:")
print(render(initial_state))

print("\nValid moves from initial state:")
for i, s in enumerate(get_valid_moves(initial_state)):
    print(f"Move {i+1}: {render(s)}")


Initial state:
●●●●○●●●●●

Valid moves from initial state:
Move 1: ●●○○●●●●●●
Move 2: ●●●●●○○●●●


### Step 2. Build the DAG of the Game

We now build the **transition graph** $\mathcal{G} = (V, E)$ of all **reachable states** starting from the **initial state** by forward exploration of valid transitions. Since each state transition is deterministic and we assume one move per time step, the graph will be a **directed acyclic graph (DAG)** rooted at the initial state.

The resulting graph will be a **dictionary** mapping each visited state to the list of its **legal successors** (reachable via one valid move), and a **set of all discovered states**, to avoid revisiting.  

* We will use a **breadth-first search (BFS)** to guarantee exploration in increasing depth.
* Each state will be represented as an immutable `tuple[int, ...]` for hashability.
* The graph will be built incrementally: if a newly generated successor hasn’t been seen before, it is added to the exploration queue.
* We will store for each state its **successor states** via valid jumps.

In [45]:
transition_graph = defaultdict(list)

# Set of all discovered states (to prevent revisiting)
discovered_states = set()
discovered_states.add(initial_state)

# Queue for breadth-first traversal
queue = deque()
queue.append(initial_state)

# Build the transition graph by exploring all reachable configurations
while queue:
    current_state = queue.popleft()
    successors = get_valid_moves(current_state)
    for next_state in successors:
        transition_graph[current_state].append(next_state)
        if next_state not in discovered_states:
            discovered_states.add(next_state)
            queue.append(next_state)

print(f"Number of reachable states: {len(discovered_states)}")
print(f"Number of transitions: {sum(len(v) for v in transition_graph.values())}")

print("\nSample transitions:")
for s in list(transition_graph.keys())[:9]:
    print(f"{render(s)} →")
    for t in transition_graph[s]:
        print(f"     {render(t)}")

Number of reachable states: 33
Number of transitions: 45

Sample transitions:
●●●●○●●●●● →
     ●●○○●●●●●●
     ●●●●●○○●●●
●●○○●●●●●● →
     ○○●○●●●●●●
     ●●○●○○●●●●
●●●●●○○●●● →
     ●●●○○●○●●●
     ●●●●●○●○○●
○○●○●●●●●● →
     ○○●●○○●●●●
●●○●○○●●●● →
     ○○●●○○●●●●
     ●●○●○●○○●●
●●●○○●○●●● →
     ●○○●○●○●●●
     ●●●○○●●○○●
●●●●●○●○○● →
     ●●●○○●●○○●
○○●●○○●●●● →
     ○○○○●○●●●●
     ○●○○○○●●●●
     ○○●●○●○○●●
●●○●○●○○●● →
     ○○●●○●○○●●
     ●●○●○●○●○○


### Step 3. Identify Terminal States

We now proceed to identify the **terminal states** in the transition graph $\mathcal{G} = (V, E)$ that we constructed by forward exploration from the initial state. Recall that a **terminal state** is one with **no valid moves**, i.e., no $(i, i+1, i+2)$ subsequence of form $(1,1,0)$ or $(0,1,1)$. Equivalently, in our transition graph, a terminal state is one with **no outgoing edges**, i.e., it has no successors in the dictionary `transition_graph`.

In [44]:
# Extract terminal states: no successors in the graph
terminal_states = {state for state in discovered_states if len(transition_graph[state]) == 0}

print(f"Number of terminal states: {len(terminal_states)}")
for i, state in enumerate(list(terminal_states)):
    print(f"{i+1}: {render(state)}  (Remaining pegs: {sum(state)})")

Number of terminal states: 7
1: ●○●○○○○○○●  (Remaining pegs: 3)
2: ○●○○○●○●○○  (Remaining pegs: 3)
3: ○○○○○○○○●○  (Remaining pegs: 1)
4: ○○○○○●○○○○  (Remaining pegs: 1)
5: ●○○●○○○●○●  (Remaining pegs: 4)
6: ○○○●○○○●○○  (Remaining pegs: 2)
7: ●○○○○●○○○●  (Remaining pegs: 3)


The set `terminal_states` is a subset of `discovered_states` with out-degree zero. We can also precompute the **reward** for each terminal state, since it is the number of remaning pegs. We also verify which terminal states have the minimum number of pegs, i.e., the maximum reward (since our reward is negative peg count).

In [43]:
# Reward function on terminal states
rewards = {state: reward(state) for state in terminal_states}

min_pegs = min(sum(s) for s in terminal_states)
optimal_value = -min_pegs

print(f"\nOptimal terminal value: {optimal_value} (i.e., {min_pegs} pegs left)")


Optimal terminal value: -1 (i.e., 1 pegs left)


### Step 4. Solve the Bellman Equation 

We now proceed to implement **value iteration** to solve the **Bellman equation** over the **DAG** defined by our transition graph. Since the environment is deterministic and acyclic, and rewards are concentrated at terminal states, the value function admits a particularly elegant structure.

For a general Markov Decision Process (MDP), the value function satisfies the Bellman equation:

$$
V(s) = \max_{a \in A(s)} \left[ R(s,a) + \sum_{s'} P(s' \mid s,a) V(s') \right].
$$

In our setting, the environment is:

* **Deterministic**: $P(s' \mid s,a) = 1$ for some $s'$.
* **Single-action per state**: no explicit action set; the set of available "moves" from a state $s$ is encoded in the graph edges.
* **Reward only at terminal states**: $R(s) = \sum_i s_i$ if $s$ is terminal,  and $R(s) = 0$ otherwise.
* **Transitions carry zero reward**: we can think of the "reward per move" as $r(s,s') = 0$.

Hence, the Bellman equation simplifies to:

$$
V(s) =
\begin{cases}
-\sum_i s_i, & \text{if } s \text{ is terminal}, \\
\max_{s' \in \mathrm{Succ}(s)} V(s'), & \text{otherwise}.
\end{cases}
$$

This is a **pure max-propagation** dynamic program: each state's value is determined by selecting the *best-valued* successor. Since the graph is a DAG, we can sort the states topologically, and propagate values backwards in a single pass. Hence, value iteration becomes equivalent to backward dynamic programming over a DAG with maximization at each node and no additive reward cost. This setup is an instance of deterministic shortest-path problems on acyclic graphs—except we are maximizing terminal reward instead of minimizing cost.

In [20]:
# Compute in-degree for each node
in_degree = defaultdict(int)
for s in transition_graph:
    for s_next in transition_graph[s]:
        in_degree[s_next] += 1

# Start with nodes that have no incoming edges
queue = deque([s for s in discovered_states if in_degree[s] == 0])
topo_order = []

while queue:
    s = queue.popleft()
    topo_order.append(s)
    for s_next in transition_graph.get(s, []):
        in_degree[s_next] -= 1
        if in_degree[s_next] == 0:
            queue.append(s_next)

# We want reverse topological order for dynamic programming
topo_order.reverse()

In [40]:
# Initialize value function V and policy π
V = {}
policy = {}

for s in discovered_states:
    if s in terminal_states:
        V[s] = rewards[s]
        policy[s] = None
    else:
        V[s] = float('-inf')
        policy[s] = None

# Value iteration in reverse topological order
for s in topo_order:
    successors = transition_graph.get(s, [])
    if successors:
        best_s = max(successors, key=lambda s_next: V[s_next])
        V[s] = V[best_s]
        policy[s] = best_s  # store optimal successor

### Step 5. Calculate Optimal Trajectory

We now reconstruct and render the optimal trajectory from the initial state using the stored policy map $\pi : \mathcal{S} \to \mathcal{S}$ computed during value iteration.

At each state $s$, the policy tells us the best successor $s' = \pi(s)$ to move to. We terminate the reconstruction when we reach a terminal state, i.e., when $\pi(s) = \text{None}$.

In [41]:
def get_trajectory(initial_state):
    """Reconstruct the optimal trajectory using the policy map"""
    trajectory = [initial_state]
    current_state = initial_state

    while policy[current_state] is not None:
        next_state = policy[current_state]
        trajectory.append(next_state)
        current_state = next_state
    
    return trajectory

In [42]:
# Visual rendering of the trajectory
print("Optimal trajectory from random initial state:\n")
states = list(transition_graph.keys())

N = len(states)
trajectory = get_trajectory(states[np.random.randint(N)])
for i, state in enumerate(trajectory):
    print(f"Step {i:2d}: {render(state)}   (Value: {V[state]})")

Optimal trajectory from random initial state:

Step  0: ○○●●○●○○●●   (Value: -1)
Step  1: ○○○○●●○○●●   (Value: -1)
Step  2: ○○○○○○●○●●   (Value: -1)
Step  3: ○○○○○○●●○○   (Value: -1)
Step  4: ○○○○○○○○●○   (Value: -1)
